## deep convolution GAN

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import random

# GPU 디바이스 목록 가져오기

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print("사용 가능한 GPU 디바이스:")
    for gpu in gpus:
        print("디바이스 이름:", gpu.name)
else:
    print("사용 가능한 GPU 디바이스가 없습니다.")

###이미지 전처리 코드
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\img_align_celeba"
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\10000"
# image_dir = "C:\\Myfolder\\AI_tools\\dfl\\df_NVIDIA_RTX3000_series\\workspace\\data_src\\aligned"
image_dir = "C:\\Myfolder\\AI_tools\\sd.webui\\this\\fun\\image"
# image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\facesetav"
# image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\picted"
crop_width, crop_height = 1024, 1024  # 자를 크기
resize_width, resize_height = 64, 64  # 변경할 크기

# 이미지 목록 가져오기
image_files = os.listdir(image_dir)

images = []
for filename in image_files:
    img = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, filename), target_size=(resize_width, resize_height))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # 이미지 정규화
    images.append(img)
images = np.array(images)


# 이미지 중에서 무작위로 하나 선택
random_index = random.randint(0, len(images) - 1)
selected_image = images[random_index]

# # # 선택한 이미지 표시
# plt.imshow(selected_image)
# plt.show()

In [ ]:
## GAN 부분
layers = tf.keras.layers
models =tf.keras.models


# 판별자 정의
# 1. 인풋정의, 합성곱층 - 리키렐루 - 드랍아웃
discriminator_input = layers.Input(shape=(resize_width, resize_height,3))
x = layers.Conv2D(64, kernel_size=4, strides=2, padding="same", use_bias= False)(discriminator_input)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)

# 2. 합성곱 - 배치노말 - 리키렐루 - 드랍아웃
x = layers.Conv2D(128, kernel_size=4, strides=2, padding="same", use_bias= False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)

# 3. 합성곱 - 배치노말 - 리키렐루 - 드랍아웃
x = layers.Conv2D(256, kernel_size=4, strides=2, padding="same", use_bias= False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)

# 4. 합성곱 - 배치노말 - 리키렐루 - 드랍아웃
x = layers.Conv2D(512, kernel_size=4, strides=2, padding="same", use_bias= False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)

# 5. 합성곱 - 시그모이드
x = layers.Conv2D(3, kernel_size=3, strides=1, padding="valid", use_bias= False, activation="sigmoid")(x)
discriminator_output = layers.Flatten()(x)
discriminator = models.Model(discriminator_input, discriminator_output)

discriminator.summary()

In [ ]:
generator_input = layers.Input(shape=(588,))
x = layers.Reshape((1, 1, 588))(generator_input)
x = layers.Conv2DTranspose(256, kernel_size=4, strides=1, padding="valid", use_bias = False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", use_bias = False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", use_bias = False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same", use_bias = False)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

generator_output = layers.Conv2DTranspose(3, kernel_size=4, strides=1, padding="same", use_bias = False, activation = 'tanh')(x)
generator = models.Model(generator_input, generator_output)

generator.summary()

In [ ]:
class DCGAN(models.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(DCGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self):
        super(DCGAN, self).compile()
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.d_optimizer = tf.optimizers.Adam(learning_rate=0.003, beta_1=0.5, beta_2=0.999)
        self.g_optimizer = tf.optimizers.Adam(learning_rate=0.003, beta_1=0.5, beta_2=0.999)
        self.d_loss_metric = tf.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(random_latent_vectors, training=True)
            real_predictions = self.discriminator(real_images, training=True)
            fake_predictions = self.discriminator(generated_images, training=True)
            real_labels = tf.ones_like(real_predictions)
            real_noisy_labels = real_labels + 0.1 * tf.random.uniform(tf.shape(real_predictions))
            fake_labels = tf.zeros_like(fake_predictions)
            fake_noisy_labels = fake_labels - 0.1 * tf.random.uniform(tf.shape(fake_predictions))
            d_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
            d_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
            d_loss = (d_real_loss + d_fake_loss) / 2.0
            g_loss = self.loss_fn(real_labels, fake_predictions)

        gradients_of_discriminator = disc_tape.gradient(d_loss, self.discriminator.trainable_variables)
        gradients_of_generator = gen_tape.gradient(g_loss, self.generator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        
        return {m.name: m.result() for m in self.metrics}

dcgan = DCGAN(discriminator=discriminator, generator=generator, latent_dim=192)
dcgan.compile()

In [ ]:
dcgan.fit(images, epochs=10)

In [ ]:
noise = np.random.randn(192)

gen = generator(np.array([noise])-0.5)
gen[0]

plt.imshow(gen[0])
plt.show()

In [ ]:
# # 가중치 저장
# dcgan.discriminator.save_weights('discriminator_weights.h5')
# dcgan.generator.save_weights('generator_weights.h5')